In [1]:
import gym
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import deque
from DQN_agent import DQN

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

%matplotlib inline 
if 'inline' in matplotlib.get_backend():
    from IPython import display

In [2]:
# create environment & define params

env = gym.make('MountainCar-v0')

In [3]:
def game_play(env, n_trials):
    '''Let the DQN agent play Mountain Car'''
    
    # list to store steps required to complete each game
    reward_list = []
    
    # create new DQN object
    dqn = DQN(env)

    for i in range(n_trials):
        state = env.reset()
        state = dqn.state_to_float(state)
        terminal = False
        cur_reward = 0
        
        while not terminal:

            # render for the last 10 episodes
            if i >= (n_trials - 5): 
                env.render()

            # choose action:
            action = dqn.choose_action(state)
            
            # find next state & reward
            next_state, reward, terminal, _ = env.step(action)
            next_state = dqn.state_to_float(next_state)
            # append to memory
            dqn.add_memory(state, action, reward, next_state, terminal)
            state = next_state
            
            if terminal:
                break
                
            cur_reward += reward
            dqn.replay_memory()
        
            # update target network
            dqn.target_update(abs(cur_reward))
                    
        reward_list.append(cur_reward)
        # update epsilon
        dqn.epsilon = dqn.epsilon_update()
        
        if i%100 == 0:
            print(f'Game {i}; reward {cur_reward}; epsilon {dqn.epsilon}')

    env.close()
        
    return reward_list

In [4]:
reward_list = game_play(env, 10)

Game 0; reward -199.0; epsilon 0.9895050000000001
